In [1]:
import pandas as pd 
import networkx as nx
import numpy as np
from functools import reduce
import datetime
import tqdm
import ast
import sys
import time

In [2]:
timetable_intercity = pd.read_csv("./data/export-intercites-gtfs-last/timetable_intercity.csv")
timetable_ter = pd.read_csv("./data/ter/timetable_ter.csv")


In [3]:
timetable = pd.concat([timetable_intercity , timetable_ter])

/Users/firasbenothman/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [4]:
timetable["after"] = timetable["arrival_time"].apply(lambda x : datetime.datetime.strptime(ast.literal_eval(x)[0],"%H:%M:%S"))
timetable["before"] = timetable["arrival_time"].apply(lambda x : datetime.datetime.strptime(ast.literal_eval(x)[-1],"%H:%M:%S"))


In [237]:
def build_graph_within_timeframe(start_time,timeframe, timetable) : 
    
    stop_2_id = {}
    
    start_time = datetime.datetime.strptime(start_time, "%H:%M:%S")
    
    timetable = timetable[timetable["before"] >= start_time]
    timetable = timetable[timetable["after" ] <= start_time + datetime.timedelta(0,timeframe*60)]
    G = nx.Graph()
    
    
    frame_indices = []
    start = time.time()
    for i in timetable.index : 
        try :           
            arrival_times = ast.literal_eval(timetable["arrival_time"][i])
            times = [datetime.datetime.strptime(arrival_times[j], "%H:%M:%S") for j in range(len(arrival_times))]
            filtered_times = [time_ for time_ in times if (time_ - start_time).seconds <= (timeframe * 60)]

            indices = np.array([times.index(time) for time in filtered_times])

            if len(indices) == 0 : continue

            frame_indices.append(i)
        
            departure_times = list(np.array(ast.literal_eval(timetable["departure_time"][i]))[indices])
            arrival_times = list(np.array(ast.literal_eval(timetable["arrival_time"][i]))[indices])
           
            stop_names = list(np.array(ast.literal_eval(timetable["stop_name"][i]))[indices])
        except ValueError : 
            #print(timetable["stop_name"][i])
            continue
            
        longitudes = list(np.array(ast.literal_eval(timetable["stop_lon"][i]))[indices])
        latitudes = list(np.array(ast.literal_eval(timetable["stop_lat"][i]))[indices])
        durations = [0.] + ast.literal_eval(timetable["durations"][i])
        
        durations = list(np.array(durations)[indices])
        for k in range(len(stop_names)-1) :  
            curr_stop = stop_names[k]
            next_stop = stop_names[k+1]
            
            if curr_stop not in stop_2_id.keys() : 
                stop_2_id[curr_stop] = len(stop_2_id.keys())
            if next_stop not in stop_2_id.keys(): 
                stop_2_id[next_stop] = len(stop_2_id.keys())
            
            
            G.add_edge(stop_2_id[curr_stop], stop_2_id[next_stop] , weight = durations[k+1],
                       departure_time = departure_times[k+1], arrival_time= arrival_times[k+1], 
                      lon_lat =(longitudes[k+1],latitudes[k+1]))

    paths = nx.shortest_path(G)
    print("Execution time {:.2f} seconds".format(time.time() - start))
    return G, paths, stop_2_id
    

In [238]:
graph, paths , stop_2_id = build_graph_within_timeframe("08:33:00", 60, timetable)

Execution time 17.60 seconds


In [240]:
graph.edges[(0,1)]

{'arrival_time': '09:17:00',
 'departure_time': '09:11:00',
 'lon_lat': (2.30809809, 49.89053246),
 'weight': 7.0}